dataset used: https://www.kaggle.com/datasets/arjunascagnetto/ptbxl-atrial-fibrillation-detection


In [ ]:
#!pip install ecg_plot
import ecg_plot as ecg_plot

In [ ]:
import numpy as np
data = np.load('/content/drive/MyDrive/datasets/ekg_dataset/ecgeq-500hzsrfava.npy')

In [ ]:
import pandas as pd
import random

df = pd.read_csv('/content/drive/MyDrive/datasets/ekg_dataset/coorteeqsrafva.csv', sep=';')

# Filter rows where the diagnosis is "AFIB"
afib_rows = df[df['diagnosi'] == 'AFIB']

# Filter rows where the diagnosis is not "AFIB"
non_afib_rows = df[df['diagnosi'] != 'AFIB']

# Extract the indexes of "AFIB" rows
afib_indexes = afib_rows.index

# Randomly select 1500 indexes from the "NON-AFIB" rows
random_non_afib_indexes = random.sample(non_afib_rows.index.tolist(), 1500)

# Combine the indexes of "AFIB" and randomly selected "NON-AFIB" rows
all_indexes = afib_indexes.union(random_non_afib_indexes)

# Create a new DataFrame with the concatenated indexes
new_df = df.loc[all_indexes]

# Specify the path for the new CSV file
new_csv_file_path = "/content/drive/MyDrive/datasets/ekg_dataset/new_data.csv"

# Write the new DataFrame to a CSV file
new_df.to_csv(new_csv_file_path, index=False)


In [ ]:
num = 0
for i in all_indexes:
  ekgS = data[i]

  ekgST = ekgS.T
  name = f"ekg_{i}"

  if i in afib_indexes:
    ecg_plot.plot(ekgST, sample_rate = 500, show_grid = False, show_separate_line=False,show_lead_name=False,columns=1)
    ecg_plot.save_as_png(name,'/content/drive/MyDrive/datasets/ekg_images_square/afib/')

  else:
    ecg_plot.plot(ekgST, sample_rate = 500, show_grid = False, show_separate_line=False,show_lead_name=False,columns=1)
    ecg_plot.save_as_png(name,'/content/drive/MyDrive/datasets/ekg_images_square/non_afib/')


In [ ]:
import os
import shutil

src_dataset_dir = '/content/drive/MyDrive/datasets/ekg_images_square/'
train_dir = '/content/drive/MyDrive/datasets/EKG/datasetOriginal/train'
test_dir = '/content/drive/MyDrive/datasets/EKG/datasetOriginal/test'
validation_dir = '/content/drive/MyDrive/datasets/EKG/datasetOriginal/validation'

os.makedirs(test_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

for folders in os.listdir(src_dataset_dir):
    disease_path = os.path.join(src_dataset_dir, folders)

    train_folders = os.path.join(train_dir, folders)
    test_folders = os.path.join(test_dir, folders)
    validation_folders = os.path.join(validation_dir, folders)
    os.makedirs(train_folders, exist_ok=True)
    os.makedirs(test_folders, exist_ok=True)
    os.makedirs(validation_folders, exist_ok=True)

    print(train_folders)

    # List all images in the breed folder
    images = os.listdir(disease_path)
    # Randomly shuffle the images
    random.shuffle(images)
    # Calculate the number of images to move to the testing dataset (20%)
    num_test_images = int(0.2 * len(images))
    num_validaton_images = int(0.2 * len(images))

    # Copy images to the testing dataset
    for image in images[:num_test_images]:
        src_image_path = os.path.join(disease_path, image)
        dst_image_path = os.path.join(test_folders, image)
        shutil.copy(src_image_path, dst_image_path)

    # copy images to the validation dataset
    for image in images[num_test_images:num_validaton_images+num_test_images]:
        src_image_path = os.path.join(disease_path, image)
        dst_image_path = os.path.join(validation_folders, image)
        shutil.copy(src_image_path, dst_image_path)

    # copy the remaining images to the training dataset
    for image in images[num_test_images:]:
        src_image_path = os.path.join(disease_path, image)
        dst_image_path = os.path.join(train_folders, image)
        shutil.copy(src_image_path, dst_image_path)

In [ ]:
#!pip install tensorflow-addons
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import optimizers
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [ ]:
img_width, img_height = 1060, 1479
batch_size = 8

train_dir = '/content/drive/MyDrive/datasets/EKG/datasetOriginal/train'
test_dir = '/content/drive/MyDrive/datasets/EKG/datasetOriginal/test'
validation_dir = '/content/drive/MyDrive/datasets/EKG/datasetOriginal/validation'

train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
base_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_vgg16.trainable = False

for layer in base_vgg16.layers[-4:]:
  layer.trainable = True

x = base_vgg16.output
x = layers.Conv2D(32, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.BatchNormalization()(x)
# x = layers.Flatten()(x)  # Flatten layer to convert 2D output to 1D
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(8, activation='relu')(x)
predictions = layers.Dense(2, activation='softmax')(x)

vgg16_model = tf.keras.models.Model(inputs = base_vgg16.input, outputs= predictions)
vgg16_model.summary()

In [ ]:
vgg16_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tfa.optimizers.AdaBelief(learning_rate=1e-3), metrics=["accuracy", tf.keras.metrics.Recall()])

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    "/content/drive/MyDrive/datasets/EKG/dataset300/model_checkpointtfaSigmoid.h5",  # Path to save the checkpoint
    save_best_only=True,    # Save only the best model
    monitor="val_loss",     # Metric to monitor (e.g., validation loss)
    mode="min",             # Minimize the monitored metric
    verbose=1                # Verbosity level
)

In [ ]:
history = vgg16_model.fit(train_generator, epochs=100, validation_data=validation_generator,callbacks=[checkpoint_callback])
vgg16_model.evaluate(test_generator)
# loss: 0.1567 - accuracy: 0.9531 - recall_2: 0.9531